In [12]:
try:
    import wandb
except
    !pip install wandb
    import wandb
    pass
!wandb login

wandb: Currently logged in as: ap-wt (use `wandb login --relogin` to force relogin)


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch.nn as nn
import torchvision
import ast
import torch
from tqdm import tqdm

# this should probably be changed to something smart, right?
KAGGLE_PATH_ANNOTATIONS = '/kaggle/input/tensorflow-great-barrier-reef/train.csv'
KAGGLE_PATH_IMG_DIR = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
LOCAL_PATH_ANNOTATIONS = 'data/train.csv'
LOCAL_PATH_IMG_DIR = 'data/train_images/'
COLAB_PATH_ANNOTATIONS = '/content/drive/MyDrive/data/train.csv'
COLAB_PATH_IMG_DIR = '/content/drive/MyDrive/data/train_images/'

# to powinno działać jako taki dict config, możemy tutaj ustawiać parametry a tak to tylko wszędzie ustawiamy wandb.config.learning_rate np, mam nadzieję,że to loguje i działa
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 2,
  "batch_size": 1,
  "momentum": 0.9,
  "weight_decay": 0.0005
}

In [15]:
class StarfishDataset(Dataset):
    def __init__(self,
                 annotations_file=COLAB_PATH_ANNOTATIONS,
                 img_dir=COLAB_PATH_IMG_DIR
                 ):
        self.img_labels = pd.read_csv(annotations_file)
        self.annotated = self.img_labels[self.img_labels['annotations'] != '[]']  # get only annotated frames
        self.img_dir = img_dir

    def __len__(self):
        return len(self.annotated)

    def __getitem__(self, idx):
        image = read_image(os.path.join(self.img_dir, 'video_{}'.format(self.annotated.iloc[idx][0]),
                                        '{}.jpg'.format(self.annotated.iloc[idx][2])))
        min_image = image.min()
        max_image = image.max()
        # normalize image to 0-1 - required by torchvision
        image -= min_image
        image = torch.FloatTensor(image/max_image)

        labels = self.annotated.iloc[idx][-1]
        labels = ast.literal_eval(labels)
        coords = []
        for parsed_label in labels:
            x1, y1 = parsed_label['x'], parsed_label['y']
            x2, y2 = x1+parsed_label['width'], y1+parsed_label['height']
            coords.append([x1, y1, x2, y2])
            
        target = [torch.FloatTensor(coords), torch.LongTensor([0 for _ in range(len(coords))])] # label has to be integer, since we have only one label I coded it as 1 for simplicity
        return image, target

# dataset = StarfishDataset()
# dataset.__getitem__(0)


In [16]:
torch.manual_seed(1)

dataset = StarfishDataset()
train_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# extract only small part of the data for faster learning / testing process
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])


print('Train dataset: {} instances, test dataset: {}'.format(len(train_dataset), len(test_dataset)))


train_dataloader = DataLoader(
    train_dataset, batch_size=wandb.config['batch_size'], shuffle=False, num_workers=1)
test_dataloader = DataLoader(
    test_dataset, batch_size=wandb.config['batch_size'], shuffle=False, num_workers=1)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
cpu = torch.device('cpu')
print('Used device: {}'.format(device))

num_classes = 2  # starfish and not starfish I guess

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=wandb.config['learning_rate'], momentum=wandb.config['momentum'], weight_decay=wandb.config['weight_decay'])
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

Train dataset: 392 instances, test dataset: 99
Used device: cuda


In [17]:
# TODO: finish eval loop
# https://pytorch.org/vision/stable/models.html#runtime-characteristics see Faster R-CNN for the details of this model, what it requires, returns, etc

wandb.init(project="great-barrier-reef", entity="ap-wt", config = wandb.config)

for e in tqdm(range(wandb.config['epochs'])):
    
    model.train()
    for images, targets in train_dataloader:
        target = []
        for i in range(len(images)):
            d = {}
            d['boxes'] = targets[0][i].to(device)
            d['labels'] = targets[1][i].to(device)
            target.append(d)

        images = images.to(device)

        loss_dict = model(images, target)
        loss = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

    model.eval()
    print('Reached eval')
    with torch.no_grad():
        groundtruth, predictions = None, None
        for idx, (images, targets) in enumerate(test_dataloader):
            # torch.cuda.empty_cache()
            images = images.to(device)
            predictions = model(images)
            if idx == 0:
                print(predictions)
            wandb.log({"scores": predictions[0]['scores']})

            # print(predictions)
            if idx % 100 == 0:
                print(predictions[0]['scores'])

    optimizer.step()
wandb.finish()

  0%|          | 0/2 [00:00<?, ?it/s]

Reached eval
[{'boxes': tensor([], device='cuda:0', size=(0, 4)), 'labels': tensor([], device='cuda:0', dtype=torch.int64), 'scores': tensor([], device='cuda:0')}]
tensor([], device='cuda:0')


 50%|█████     | 1/2 [06:06<06:06, 366.22s/it]

Reached eval
[{'boxes': tensor([], device='cuda:0', size=(0, 4)), 'labels': tensor([], device='cuda:0', dtype=torch.int64), 'scores': tensor([], device='cuda:0')}]
tensor([], device='cuda:0')


100%|██████████| 2/2 [12:12<00:00, 366.08s/it]
